Simplified inference without using the [dequantization "trick"](DequantizeTrick.ipynb)

In [1]:
!pip install trl transformers accelerate git+https://github.com/huggingface/peft.git -Uqqq
!pip install datasets bitsandbytes einops -Uqqq

In [2]:
import torch
from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments, GenerationConfig
from peft import LoraConfig, get_peft_model, PeftConfig, PeftModel, prepare_model_for_kbit_training
from trl import SFTTrainer
import warnings
warnings.filterwarnings("ignore")

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

In [3]:
tokenizer = AutoTokenizer.from_pretrained('nihiluis/finadv100')
model = AutoModelForCausalLM.from_pretrained('abhishek/llama-2-7b-hf-small-shards', torch_dtype=torch.float16, device_map="auto") #Base_Model for example :meta-llama/Llama-2-13b-chat-hf
model = PeftModel.from_pretrained(model, 'nihiluis/finadv100', device_map="auto")

Loading checkpoint shards:   0%|          | 0/10 [00:00<?, ?it/s]

In [4]:
input_context = """
### Human
So, I made a paypal account and set my age as 18 when I was like 11. (I'm 15 now.) I didn't enter any information and only used it to put gift cards onto. When I was 13, I added my card to it that I got with BOFA. Then I signed up for a website subscription trial, and canceled it before it charged me, except that it didn't cancel because my service cut so they charged me the 30$. Afterwards, I realized and canceled it properly, but I wasn't aware I had a limited amount of time to pay it, or that I had to pay it at all. This was in March. Today I woke up to a letter from ACI (paypal's debt collection agency I think??) That says I have to pay it by September 28th, but my mom refuses to help me and won't explain any of it, telling me that this is a good time to learn on my own because she had to at 18, and that she won't explain it, and I don't understand how or what I'm supposed to do. Paypal locked my account so I can't log in to pay it. 've been freaking out about this for hours, what should I do?

### Assistant
"""

input_ids = tokenizer.encode(input_context, return_tensors="pt").to("cuda")
output = model.generate(inputs=input_ids, max_length=1024, temperature=0.3, num_return_sequences=1)

generated_text = tokenizer.decode(output[0], skip_special_tokenz=True)
generated_text

"<s> \n### Human\nSo, I made a paypal account and set my age as 18 when I was like 11. (I'm 15 now.) I didn't enter any information and only used it to put gift cards onto. When I was 13, I added my card to it that I got with BOFA. Then I signed up for a website subscription trial, and canceled it before it charged me, except that it didn't cancel because my service cut so they charged me the 30$. Afterwards, I realized and canceled it properly, but I wasn't aware I had a limited amount of time to pay it, or that I had to pay it at all. This was in March. Today I woke up to a letter from ACI (paypal's debt collection agency I think??) That says I have to pay it by September 28th, but my mom refuses to help me and won't explain any of it, telling me that this is a good time to learn on my own because she had to at 18, and that she won't explain it, and I don't understand how or what I'm supposed to do. Paypal locked my account so I can't log in to pay it. 've been freaking out about thi